<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>模型</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#my-notebook-1" data-toc-modified-id="my-notebook-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>my notebook 1</a></span><ul class="toc-item"><li><span><a href="#External-HDD,-BUP-Slim-BK-+-USB3.0" data-toc-modified-id="External-HDD,-BUP-Slim-BK-+-USB3.0-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>External HDD, BUP Slim BK + USB3.0</a></span></li></ul></li><li><span><a href="#my-notebook-2" data-toc-modified-id="my-notebook-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>my notebook 2</a></span></li><li><span><a href="#my-desktop-2" data-toc-modified-id="my-desktop-2-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>my desktop 2</a></span><ul class="toc-item"><li><span><a href="#External-HDD,-BUP-Slim-BK-+-USB3.0" data-toc-modified-id="External-HDD,-BUP-Slim-BK-+-USB3.0-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>External HDD, BUP Slim BK + USB3.0</a></span></li></ul></li></ul></li></ul></div>

# 模型

我们使用CIFAR10数据集。

In [1]:
# % https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(32, 128, 3, 1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, 1, padding=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(32*32//4*256, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    
    
#     import torch.autograd.profiler as profiler
#     with profiler.profile(record_shapes=True) as prof:
#         with profiler.record_function("model_inference"):

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                     epoch, batch_idx * len(data), len(train_loader.dataset),
#                     100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

#     print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main(batch_size=32,test_batch_size=32):
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.CIFAR10 ('../data', train=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10 ('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")



In [2]:
import platform
import numpy as np
import sys
import psutil
from torchvision import datasets


def list_env():
    print(platform.node())
    print(platform.platform())
    print(platform.machine())
    try:
        import cpuinfo
        print(cpuinfo.get_cpu_info()["brand_raw"])
        print("cpu-count", cpuinfo.get_cpu_info()["count"])
    except:
        print(platform.processor())
        print("cpu-count", psutil.cpu_count())
        
    print(sys.version)
    print(torch.__version__)
    print(torch.__config__.show())
    print(torch.__config__.parallel_info())

    
    print("gpu available:", torch.cuda.is_available())
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        
        
def model_summary():            
    model = Net()
    for param in model.parameters():
        print(tuple(param.size()), param.dtype,
              np.array(param.size()).prod()*4/(1024*1024))

def download_data():
    datasets.CIFAR10('../data', download=True)
    
def benchmark(try_gpu=True, try_cpu=False):

    if try_gpu and torch.cuda.is_available():        
        print("device=gpu")
        psutil.cpu_percent(percpu=True)
        argv = sys.argv
        sys.argv = ["", "--epochs", "1"]
        %timeit -r1 -n1 main()
        sys.argv = argv
        print("cpu-percent:", psutil.cpu_percent(percpu=True))
        print()
        
    if try_cpu:
        print("device=cpu")
        psutil.cpu_percent(percpu=True)
        argv = sys.argv
        sys.argv = ["", "--epochs","1", "--no-cuda"]
        %timeit -r1 -n1 main()
        sys.argv = argv
        print("cpu-percent:", psutil.cpu_percent(percpu=True))
        print()
    
model_summary()
download_data()

(32, 3, 3, 3) torch.float32 0.0032958984375
(32,) torch.float32 0.0001220703125
(128, 32, 3, 3) torch.float32 0.140625
(128,) torch.float32 0.00048828125
(256, 128, 3, 3) torch.float32 1.125
(256,) torch.float32 0.0009765625
(512, 65536) torch.float32 128.0
(512,) torch.float32 0.001953125
(128, 512) torch.float32 0.25
(128,) torch.float32 0.00048828125
(10, 128) torch.float32 0.0048828125
(10,) torch.float32 3.814697265625e-05
Files already downloaded and verified


# Results

## my notebook 1

In [3]:
list_env()

DESKTOP-U61NOQ2
Windows-10-10.0.18362-SP0
AMD64
Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
cpu-count 8
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.7.1
PyTorch built with:
  - C++ Version: 199711
  - MSVC 192729112
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191125 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 2019
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_37,code=compute_37
  - CuDNN 7.6.5
  - Magma 2.5.4
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -openmp:experimental -DNDEBUG -DUSE_FBGEMM -DUSE_VULKAN_WRAPPER, PERF

### External HDD, BUP Slim BK + USB3.0

cuda is running 100%

In [3]:
benchmark()
benchmark()

device=gpu
Test set: Average loss: 1.1687, Accuracy: 5827/10000 (58%)
4min 36s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [19.9, 3.2, 11.9, 4.4, 8.6, 5.6, 5.9, 71.6]

device=gpu
Test set: Average loss: 1.1561, Accuracy: 6020/10000 (60%)
4min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [12.5, 2.3, 7.7, 4.8, 3.5, 2.1, 3.3, 80.6]



## my notebook 2

In [ ]:
benchmark()

## my desktop 2

In [3]:
list_env()

DESKTOP-0J5HGI3
Windows-10-10.0.19041-SP0
AMD64
AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD
cpu-count 16
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
1.7.1
PyTorch built with:
  - C++ Version: 199711
  - MSVC 192729112
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191125 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 2019
  - CPU capability usage: AVX2
  - CUDA Runtime 11.0
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_37,code=compute_37
  - CuDNN 8.0.4
  - Magma 2.5.4
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -openmp:experimental -DNDEBUG -DUS

### External HDD, BUP Slim BK + USB3.0
cuda is runnig 98%.

In [3]:
benchmark()
benchmark()

device=gpu
Test set: Average loss: 1.1820, Accuracy: 5897/10000 (59%)
39.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [8.3, 1.2, 3.8, 6.6, 2.2, 10.7, 6.5, 9.7, 4.4, 2.2, 4.0, 1.2, 2.3, 5.6, 2.2, 80.2]

device=gpu
Test set: Average loss: 1.1820, Accuracy: 5897/10000 (59%)
37.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
cpu-percent: [7.6, 1.7, 3.0, 10.9, 6.8, 3.1, 3.7, 5.9, 1.9, 2.4, 6.5, 3.1, 3.6, 6.3, 2.2, 87.5]

